<center> <font size = 24 color = 'steelblue'> <b>Machine Translation<br>


![Image Description](https://labcontent.simplicdn.net/data-content/content-assets/Data_and_AI/NLP_November/Lesson%205/Machine%20translation.jpg)

## Overview:

The goal is to build a machine translation pipeline by leveraging embeddings to translate an English dictionary to French. It involves loading necessary libraries and embeddings, working with embedding vectors, and using cosine similarity to measure semantic similarity. Additionally, gradient computation optimizes the transformation matrix for effective translation.

# <a id= 'f0'>
<font size = 4>
    
**Table of Contents:**<br>
[1. Introduction](#f1)<br>
[2. Loading libraries](#f2)<br>
[3. Loading embeddings](#f3)<br>
[4. Translating English dictionary to French](#f4)<br>
> [4.1 Working with embeddings](#f4.1)<br>
> [4.2 Computing the gradient of loss in respect to transform matrix R](#f4.2)<br>
[3. Cosine Similarity](#f3)<br>

##### <a id = 'f1'>
<font size = 10 color = 'midnightblue'> **Introduction**

<div class="alert alert-block alert-success">
<font size = 4>

- Machine translation involves the use of automated systems to translate text or speech from one language to another.
- NLP plays a crucial role in understanding, interpreting, and generating human language in a way that considers context and meaning.
- NLP techniques are employed to enhance the quality and accuracy of machine translation systems.
- NLP helps in addressing linguistic nuances, context understanding, and idiosyncrasies specific to each language.

##### <a id = 'f2'>
<font size = 10 color = 'midnightblue'> **Load the Libraries**

In [2]:
# # install below libraries if not done
# !pip install numpy==1.23.5
# !pip install tensorflow==2.13.1
# !pip install nltk==3.8.1
# !pip install pandas==1.5.3
# !pip install gensim==4.2.0
# !pip install scipy==1.9.3
# !pip install matplotlib==3.6.3
# !pip install scikit-learn==1.3.1

In [3]:
# !pip install gensim

In [4]:
import nltk
import pdb
import pickle
import string
import pandas as pd
import time
import gensim
import matplotlib.pyplot as plt
import numpy as np
import scipy

from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer
import re
from nltk.stem import PorterStemmer

from sklearn.metrics.pairwise import cosine_similarity

In [5]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /voc/work/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /voc/work/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [6]:
twitter_samples.fileids()

['negative_tweets.json', 'positive_tweets.json', 'tweets.20150430-223406.json']

In [7]:
data = twitter_samples.strings('positive_tweets.json')
# data

##### <a id = 'f3'>
<font size = 10 color = 'midnightblue'> **Load English and French Embeddings**

In [8]:
en_emb_subset = pickle.load(open("en_embeddings.p", 'rb'))
fr_emb_subset = pickle.load(open("fr_embeddings.p", 'rb'))

In [9]:
file =  pd.read_csv('en-fr.train.txt', delimiter = ' ', header =None, index_col = [0]).squeeze('columns')
eng_to_fr_dict_train =  file.to_dict()

In [10]:
file.head()

0
the     le
the    les
the     la
and     et
was    fut
Name: 1, dtype: object

In [11]:
eng_to_fr_dict_train

{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
 'that': 'cela',
 'with': 'avec',
 'from': 'depuis',
 'this': 'ce',
 'utc': 'tuc',
 'his': 'son',
 'not': 'pas',
 'are': 'sont',
 'talk': 'parlez',
 'which': 'lequel',
 'also': 'egalement',
 'were': 'étaient',
 'but': 'mais',
 'have': 'ont',
 'one': 'one',
 'new': 'nouveautés',
 'first': 'premiers',
 'page': 'page',
 'you': 'you',
 'they': 'eux',
 'had': 'avais',
 'article': 'article',
 'who': 'who',
 'all': 'all',
 'their': 'leurs',
 'there': 'là',
 'made': 'fabriqué',
 'its': 'son',
 'people': 'personnes',
 'may': 'peut',
 'after': 'aprés',
 'other': 'autres',
 'should': 'devrais',
 'two': 'deux',
 'score': 'partition',
 'her': 'her',
 'can': 'peut',
 'would': 'ferait',
 'more': 'plus',
 'she': 'elle',
 'when': 'quand',
 'time': 'heure',
 'team': 'equipe',
 'american': 'américains',
 'such': 'telles',
 'discussion': 'débat',
 'links': 'liens',
 'only': 'seule',
 'some': 'quelques',
 'see': 'vois',
 'united': 'unies',
 'year

In [12]:
len(eng_to_fr_dict_train)

5000

In [29]:
file2 =  pd.read_csv('en-fr.test.txt', delimiter = ' ', header =None, index_col = [0]).squeeze('columns')
eng_to_fr_dict_test =  file2.to_dict()

In [14]:
file2.head()

0
torpedo       torpille
torpedo      torpilles
giovanni      giovanni
chat          discuter
chat        discussion
Name: 1, dtype: object

In [15]:
len(en_emb_subset)

6370

[top](#f0)

##### <a id= 'f4'>
<font size = 10 color = 'midnightblue'> **Translating English Dictionary to French** <br>


##### <a id = 'f4.1'>
<font size = 6 color = 'pwdrblue'> <b>Working with embeddings

<div class="alert alert-block alert-success">
<font size = 4>
    
- Generate a matrix where where the columns are the English embeddings.
- Generate a matrix where the columns correspond to the French embeddings.
- Generate the projection matrix that minimizes the F norm ||X R -Y||^2.

> - The goal is often to find a transformation matrix that minimizes the difference between two matrices.
> - The Frobenius norm is a way to measure the "size" or magnitude of a matrix.

In [30]:
# get the set of words of English

eng_words = en_emb_subset.keys()
fr_words = fr_emb_subset.keys()

<font size = 5 color = 'seagreen'> <b>Check whether embedding is present for both the English and French words present in translations dictionary

In [32]:
# eng_to_fr_dict_train

In [35]:
# eng_words
# fr_words

In [19]:
eng_emb =[]
frnch_emb = []

for eng, fr in eng_to_fr_dict_train.items(): # eng_to_fr_dict_train has english words as key & french translation as pair (i.e. 'the': 'la')
    if (eng in eng_words) and (fr in fr_words):
       # get the embeddings and store
        eng_emb.append(en_emb_subset[eng]) # en_emb_subset has word english word embeddings
        frnch_emb.append(fr_emb_subset[fr]) # fr_emb_subset has word french word embeddings

In [37]:
# eng_emb[:2]

In [40]:
len(eng_emb[0])

300

<font size = 5 color = 'seagreen'> <b>Create English and French Embedded Matrix

In [22]:
X = np.vstack(eng_emb) # stack vectors vertically to creat 2D array
X.shape # 4932 vectors of length 300

(4932, 300)

In [23]:
Y = np.vstack(frnch_emb)
Y.shape # 4932 vectors of length 300

(4932, 300)

<font size = 5 color = 'seagreen'> <b>Translation

<div class="alert alert-block alert-success">
<font size = 4>
    
The loss function will be squared Frobenius norm of the difference between
matrix and its approximation, divided by the number of training examples $m$.
</div>

<font size = 5>
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$


<font size = 4>
    
<center> where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.

##### <a id = 'f4.2'>
<font size = 6 color = 'pwdrblue'> <b>Computing the gradient of loss in respect to transform matrix R

<div class="alert alert-block alert-success">
<font size = 4>
    
* Calculate the gradient of the loss with respect to transform matrix `R`.
* The gradient is a matrix that encodes how much a small change in `R`
affect the change in the loss function.
* The gradient gives us the direction in which we should decrease `R`
to minimize the loss.
* $m$ is the number of training examples (number of rows in $X$).
* The formula for the gradient of the loss function $𝐿(𝑋,𝑌,𝑅)$ is:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$



[top](#f0)

The below code implements a simple gradient descent algorithm to optimize a transformation matrix R for minimizing the mean squared error (MSE) between the transformed input data X @ R and the target data Y.

#### Initialization:

- A random matrix R of shape (X.shape[1], X.shape[1]).
- Number of training steps (train_steps = 600) and learning rate (learning_rate = 0.8).

#### Training Loop:

- For every even iteration, the code computes the loss (MSE) and prints it.
- The gradient of the loss with respect to R is computed and used to update R in the direction that reduces the loss.
This process iteratively adjusts R to minimize the difference between the predicted and actual values in Y.

In [24]:
# R is the transformation matrix you want to learn that maps English embeddings to French embeddings.

R =  np.random.rand(X.shape[1], X.shape[1]) # random matrix
train_steps =600
learning_rate  = 0.8

for i in range(train_steps+1):
    if i%2 ==0: # for every even iteration
        diff = (X @ R) -Y # difference between transformed input and target
        sq_diff = diff**2
        loss = np.sum(sq_diff)/X.shape[0] # MSE
        print(f"loss at iteration {i} is : {loss:.3f}")
    gradient =  np.dot(X.transpose(),np.dot(X,R) -Y)*(2/X.shape[0]) # gradient of loss with respect to R
    R = R - learning_rate*gradient # update R

loss at iteration 0 is : 976.479
loss at iteration 2 is : 598.113
loss at iteration 4 is : 473.888
loss at iteration 6 is : 387.579
loss at iteration 8 is : 323.466
loss at iteration 10 is : 273.740
loss at iteration 12 is : 234.039
loss at iteration 14 is : 201.702
loss at iteration 16 is : 174.976
loss at iteration 18 is : 152.640
loss at iteration 20 is : 133.805
loss at iteration 22 is : 117.805
loss at iteration 24 is : 104.125
loss at iteration 26 is : 92.363
loss at iteration 28 is : 82.198
loss at iteration 30 is : 73.376
loss at iteration 32 is : 65.686
loss at iteration 34 is : 58.958
loss at iteration 36 is : 53.052
loss at iteration 38 is : 47.851
loss at iteration 40 is : 43.256
loss at iteration 42 is : 39.185
loss at iteration 44 is : 35.570
loss at iteration 46 is : 32.352
loss at iteration 48 is : 29.479
loss at iteration 50 is : 26.910
loss at iteration 52 is : 24.608
loss at iteration 54 is : 22.541
loss at iteration 56 is : 20.680
loss at iteration 58 is : 19.004
lo

In [25]:
R

array([[-1.07557035e-02, -1.13599228e-02, -1.27648490e-06, ...,
         3.80846387e-03, -3.88543168e-03,  2.82907512e-03],
       [ 1.77251627e-02,  6.35580395e-05,  3.68668717e-03, ...,
         2.30447181e-02, -2.51617081e-03,  3.81739414e-04],
       [ 4.82764362e-03, -8.32731872e-04,  1.48547648e-02, ...,
        -4.72242601e-03, -6.94708374e-03,  3.74847059e-03],
       ...,
       [-1.95581827e-03,  4.43160135e-03, -1.83241847e-02, ...,
        -1.09451888e-02,  6.40203679e-03,  1.74951656e-03],
       [ 4.19455279e-03, -6.80765337e-03, -7.90803434e-04, ...,
        -7.29151410e-03, -1.52966042e-03, -7.95843559e-04],
       [ 7.02209931e-03, -1.24122277e-02, -4.06979709e-03, ...,
        -1.24662792e-02,  8.80968945e-03, -3.41010797e-03]])

[top](#f0)

In [26]:
pred = np.dot(X,R) # finding dot product between X and R

In [27]:
pred

array([[-0.0234479 , -0.00159961, -0.02609108, ...,  0.02847487,
         0.01677922, -0.00371091],
       [-0.0137841 , -0.00059437, -0.05508738, ...,  0.01720859,
         0.04562487,  0.00700303],
       [-0.01280458,  0.01513591, -0.01336364, ...,  0.02519028,
         0.0495907 , -0.01520748],
       ...,
       [ 0.02125568, -0.05357374, -0.03786858, ..., -0.0150685 ,
         0.03587862, -0.02819521],
       [-0.01857506, -0.00594681,  0.02864552, ...,  0.06878331,
        -0.00158362, -0.03221478],
       [-0.02692281, -0.01827884, -0.02009246, ...,  0.04388251,
         0.05717338,  0.0186611 ]])

The provided code checks how well a transformation matrix R maps input data X to target data Y using cosine similarity.

- cosine_similarity(): Calculates how similar two vectors are.

- nearest_neighbor(): Finds the vector in a list (candidates) that is most similar to a given vector v.

- test_vocab():

>> - Transforms X using the matrix R.
>> - For each transformed vector, it finds the closest match in Y.
>> - Measures the accuracy by checking if the closest match is correct.

In [28]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

def nearest_neighbor(v, candidates, k=1):
    return np.argsort([cosine_similarity(v,row) for row in candidates])[-k:]


# check if closest french embedding for each transformed eng embedding is the correct corresponding french word
def test_vocab(X,Y,R):
    pred = np.dot(X,R) # transform eng embeddigns into french
    return sum([nearest_neighbor(row,Y)==index for index, row in enumerate(pred)])/len(pred)

test_vocab(X,Y,R)

array([0.55352798])